In [1]:
from bs4 import BeautifulSoup
import re

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import pandas as pd


# driver = webdriver.Chrome()
driver = webdriver.Edge()

In [2]:
# driver = webdriver.Chrome()
driver = webdriver.Edge()

driver.get("https://fundresearch.fidelity.com/fund-screener/results/table/overview/averageAnnualReturnsYear3/desc/1?assetClass=&category=&order=assetClass%2Ccategory")
time.sleep(3)

soup = BeautifulSoup(driver.page_source, "html.parser")
table = soup.find("table", id="scrollable-results-table")


In [25]:
data = []

for tr in table.find("tbody", id="results-tbody").find_all("tr"):
    td = tr.find_all("td") 
    if len(td) > 2:
        match = re.match(r"^(.*?)\s*\((.*?)\)$", td[4].text.strip())
        name = match.group(1)
        symbol = match.group(2)
        gross_er = td[13].text.strip()
        data.append({"name": name, "symbol": symbol, "er": gross_er})


In [26]:
data

[{'name': 'Bitcoin ProFund Investor Class', 'symbol': 'BTCFX', 'er': '1.31%'},
 {'name': 'Vest Bitcoin Strategy Managed Volatility Fund Investor Class Shares',
  'symbol': 'BTCLX',
  'er': '3.44%'},
 {'name': '$Vest Bitcoin Strategy Managed Volatility Fund Institutional Class Shares',
  'symbol': 'BTCVX',
  'er': '2.96%'},
 {'name': 'Allspring Precious Metals Fund - Class A',
  'symbol': 'EKWAX',
  'er': '1.17%'},
 {'name': '$Allspring Precious Metals Fund - Class C',
  'symbol': 'EKWCX',
  'er': '1.92%'},
 {'name': 'Fidelity® Select Semiconductors Portfolio',
  'symbol': 'FSELX',
  'er': '0.62%'},
 {'name': 'OCM Gold Fund Atlas Class', 'symbol': 'OCMAX', 'er': '1.88%'},
 {'name': '$VanEck International Investors Gold Fund Class Y',
  'symbol': 'INIYX',
  'er': '1.13%'},
 {'name': '$OCM Gold Fund Investor Class', 'symbol': 'OCMGX', 'er': '2.38%'},
 {'name': 'VanEck International Investors Gold Fund Class A',
  'symbol': 'INIVX',
  'er': '1.42%'},
 {'name': 'American Century Global Gold

'Bitcoin ProFund Investor Class (BTCFX)'

'1.31%'